In [8]:
import os
import datetime

from djia_stock_prediction import Dataset
from djia_stock_prediction import TradingDaysHelper

from djia_stock_prediction import LinearRegressionStockForecaster
from djia_stock_prediction import ArimaStockForecaster
from djia_stock_prediction import ProphetStockForecaster
from djia_stock_prediction import LongShortTermMemoryStockForecaster

## Creating a dataset used for production

In [9]:
production_dataset_filename = os.path.join('data', 'djia_prod.pkl')

In [10]:
# Loading the oldest available dataset
ds = Dataset.loadDatasetFromFile(os.path.join('data', 'djia_20140324-20190322.pkl'))
# Updating the dataset with the most recent data available at the moment
ds.updateData()
# Saving the dataset to be used as the production dataset
ds.saveDataToFile(production_dataset_filename)

'data/djia_prod.pkl'

## Retrieving and updating the production dataset

In [11]:
# Loading the production dataset
ds = Dataset.loadDatasetFromFile(production_dataset_filename)

In [12]:
# If necessary
# Updating the production dataset ...
ds.updateData()
# ... and saving it with the most recent data
ds.saveDataToFile(production_dataset_filename)

'data/djia_prod.pkl'

In [13]:
ticker_symbol = 'AAPL'

In [14]:
trading_days_helper = TradingDaysHelper()

## Creating and training ML models

In [16]:
training_end = datetime.datetime.today()

Linear Regresion

In [17]:
linear_regresion_model = LinearRegressionStockForecaster(ticker_symbol, trading_days_helper=trading_days_helper)
linear_regresion_model.train(ds, end_date=training_end)
linear_regresion_model_file = linear_regresion_model.saveToFile(
    os.path.join('models', ticker_symbol + '_LinearRegression.pkl'))

In [18]:
arima_model = ArimaStockForecaster(ticker_symbol, trading_days_helper=trading_days_helper)
arima_model.train(ds, end_date=training_end)
arima_model_file = arima_model.saveToFile(os.path.join('models', ticker_symbol + '_ARIMA.pkl'))

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=5537.755, BIC=5563.413, Fit time=8.006 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=6323.819, BIC=6334.082, Fit time=0.120 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=6016.714, BIC=6037.241, Fit time=1.834 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=5537.283, BIC=5557.810, Fit time=8.051 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=5538.910, BIC=5564.568, Fit time=8.421 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=6325.712, BIC=6341.107, Fit time=0.423 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=5538.877, BIC=5564.535, Fit time=28.015 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=5539.575, BIC=5570.365, Fit time=44.581 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=5535.513, BIC=5550.908, Fit time=5.226 seconds
Fit ARIMA: order=(0, 1, 0)

In [19]:
prophet_model = ProphetStockForecaster(ticker_symbol, trading_days_helper=trading_days_helper)
prophet_model.train(ds, end_date=training_end)
prophet_model_file = prophet_model.saveToFile(os.path.join('models', ticker_symbol + '_Prophet.pkl'))

/usr/local/lib/python3.5/dist-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()


In [20]:
lstm_model = LongShortTermMemoryStockForecaster(ticker_symbol, trading_days_helper=trading_days_helper)
lstm_model.train(ds, end_date=training_end)
lstm_model_file = lstm_model.saveToFile(os.path.join('models', ticker_symbol + '_LSTM.pkl'))

Epoch 1/2


InternalError: Blas GEMM launch failed : a.shape=(1, 50), b.shape=(50, 50), m=1, n=50, k=50
	 [[Node: lstm_1/while/MatMul_4 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_1/while/Identity_2, lstm_1/while/MatMul_4/Enter)]]
	 [[Node: loss/mul/_119 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_3485_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'lstm_1/while/MatMul_4', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlet

In [ ]:
prediction_start = str_to_datetime('2019-04-01')
prediction_end = str_to_datetime('2019-09-30')

fig = plt.figure(figsize=(18, 21))
n_rows = len(models)

i = 1
for model in models:
    ax = fig.add_subplot(n_rows, 1, i)
    i = i + 1

    # Getting the predicted values
    preds = model.predict(prediction_start, to_date=prediction_end, base_dataset=ds)

    # Ploting the graph to compare the predicted closing prices against the ones in the validation set
    existing_df = ds.getDataframe(ticker_symbol=ticker_symbol, from_date=str_to_datetime('2018-04-01'));
    ax.plot(existing_df.date, existing_df.close, label='Existing Data')
    ax.plot(preds.date, preds.predicted_price, label='Prediction')
    ax.set_title('Prediction of ' + ticker_symbol + ' using ' + model.long_name)
    ax.legend()

# Saving the plot with graphs comparing predictions against validation closing prices
plt.savefig(os.path.join('figures', 'prediction_example.png'))
plt.show()